In [5]:
import tkinter as tk
import random

# liu
class Grid:
    """A superclass of all grids in the map."""
    def __init__(self, number, grid_type=None):
        self.number = number # The number of the grid.
        self.type = grid_type

class BossGrid(Grid):
    """A subclass of gird contained boss."""
    def __init__(self, number):
        super().__init__(number, "Boss")
        # Make sure boss has a suitable power.
        self.power = int(number * random.uniform(2.4, 3.6)) 

class GameGrid(Grid):
    """A subclass of gird contained game."""
    def __init__(self, number):
        super().__init__(number, "Game")
        card_game = CardDeck()
        # Randomly choose a game for the Grid.
        self.game = random.choice([f"{random.choice(card_game.card_challenges)}", "Game2", "Game3"]) 

class PointGrid(Grid):
    """A subclass of gird where player can collect points."""
    def __init__(self, number):
        super().__init__(number, "Point")
        # Edited by 5554570.
        # 20% chance of generating a high point value.
        self.point = int(number * random.uniform(1.4, 1.7)) if random.random() < 0.2 else (number // 7 + 1)
        # Edited by 5554570.
# liu

# 5554570 - beggining.
class Player:
    """Represents the player in the game."""
    def __init__(self, health_points, combat_effectiveness):
        self.health_points = health_points
        self.combat_effectiveness = combat_effectiveness
        self.player_position = 0

    def move(self, steps):
        # Move the player position without exceeding the grid limit.
        self.player_position += steps
        self.player_position = min(self.player_position, len(grids) - 1)

    def update_hp(self, hp_change):
        # Updating the players health points and making sure it doesn't go below zero.
        self.health_points += hp_change
        self.health_points = max(0, self.health_points)
    
    def update_ce(self, ce_change):
        # Updating the players combat effectiveness and making sure it doesn't go below zero.
        self.combat_effectiveness += ce_change
        self.combat_effectiveness = max(0, self.combat_effectiveness)

# Creating the card deck Game.
class CardDeck:
    def __init__(self):
        # Defining the deck of cards and card challenge variables.
        self.suits = ["Hearts", "Diamonds", "Clubs", "Spades"]
        self.values = ["Ace", "2", "3", "4", "5", "6", "7", "8", "9", "Jack", "Queen", "King"]
        self.deck = [f"{value} of {suit}" for suit in self.suits for value in self.values]
        self.red_cards = [f"{card}" for card in self.deck if "Hearts" in card or "Diamonds" in card]
        self.black_cards = [f"{card}" for card in self.deck if "Clubs" in card or "Spades" in card]
        self.odd_cards = [f"{card}" for card in self.deck if card[0] in ["Ace", "3", "5", "7", "9", "Queen"]]
        self.even_cards = [f"{card}" for card in self.deck if card[0] in ["2", "4", "6", "8", "Jack", "King"]]
        # Defining the card challenges.
        self.card_challenges = [ "Draw a red card", "Draw a black card", "Draw a odd", "Draw a even"]

    def draw_card(self):
        """Drawing a random card feom the deck."""
        return random.choice(self.deck)

    def card_in_deck(self, challenge, card):
        """Check if the card is in the deck."""
        # Using a linear searching algorithm to check if the card is in the deck.
        if "red" in challenge:
            for i in self.red_cards:
                if i == card:
                    return True
        if "black" in challenge:
            for i in self.black_cards:
                if i == card:
                    return True
        if "odd" in challenge:
            for i in self.odd_cards:
                if i == card:
                    return True
        if "even" in challenge:
            for i in self.even_cards:
                if i == card:
                    return True
        return False # Return False if the card is not in the deck.
    
def roll_dice():
    """Simulates rolling a die."""
    return random.randint(1, 6)
# 5554570 - end.

# liu
def generate_grid(number):
    """Generates a grid based on the giver number."""
    if number == 0:     # starting point. 
        return Grid(number, "Start")
    elif number == 48:  # ending point.
        return Grid(number, "End")
    # Edited by 5554570.
    # Making sure that there wouldn't be any boss battles in the first 3 squares.
    elif number < 4:    
        # 70% chance it is a point grid.
        return PointGrid(number) if random.randint(1, 10) <= 7 else GameGrid(number)
    # Edited by 5554570
    # Generate a number to choose grid according to fixed probabilities.
    else:         
        r = random.randint(1, 10)   
        if r <= 5:
            return PointGrid(number)    # Probability 50%.
        elif r <= 9:
            return GameGrid(number)     # Probability 40%.
        else:
            return BossGrid(number)     # Probability 10%.
    
# Use a list to storage grids.
grids = [generate_grid(i) for i in range(49)]   

def create_grid(canvas, grid, x, y, player_position):
    """Draw a grid in canvas."""
    canvas.create_text(x+90, y+10, text=str(grid.number+1))
    canvas.create_rectangle(x, y, x+100, y+100)
    # Draw different types of grids with specific attributes.
    if grid.type == "Point":
        canvas.create_text(x+50, y+50, fill = "blue", text="Point\n"+str(grid.point))
    elif grid.type == "Game":
        canvas.create_text(x+50, y+50, fill = "orange", text=grid.game)
    elif grid.type == "Boss":
        canvas.create_text(x+50, y+50, fill = "red", text="Boss\n"+str(grid.power))
    elif grid.type == "Start":
        canvas.create_text(x+50, y+50, text="Start")
    elif grid.type == "End":
        canvas.create_text(x+50, y+50, text="End")
    # Shows the players position.
    if grid.number == player_position:
        canvas.create_oval(x+30, y+30, x+70, y+70, fill="green")
# liu

# 5554570 - beggining.
def roll_and_move():
    """Rolls the dice, moves the player, and checks for challenges."""
    global dice_result, challenges_text # Make the widgets global so they can be updated.
    steps = roll_dice()
    player.move(steps)
    update_grid()
    dice_result.config(text=f"Dice Roll: {steps}")

    current_grid = grids[player.player_position]
    if isinstance(current_grid, PointGrid):
        # Handling the point change.
        player.update_hp(current_grid.point)
        hp_label.config(text=f"HP: {player.health_points}")
        # Update the challenge list text with the challenge and outcome.
        challenges_text.delete(1.0, tk.END)
        challenges_text.insert(tk.END, f"Points gained: {current_grid.point}\n")

    if isinstance(current_grid, BossGrid):
        # Handling the boss challenge.
        power = grids[player.player_position].power
        # Method for the boss fight (not implemented yet).
        ####challenge_success = boss_fight(power).####
        challenges_text.delete(1.0, tk.END)
        challenges_text.insert(tk.END, f"Boss Power: {power}\n")
        challenges_text.insert(tk.END, f"Player Power: {player.combat_effectiveness}\n")
        #challenges_text.insert(tk.END, f"Success!\n" if challenge_success else f"Failed!\n")
        

    if isinstance(current_grid, GameGrid) and "Draw" in current_grid.game:
        # Handling the card challenge.
        card_game = CardDeck()
        drawn_card = card_game.draw_card()
        challenge_success = card_game.card_in_deck(current_grid.game, drawn_card)
        challenges_text.delete(1.0, tk.END)   # Clear the previous challenges.
        challenges_text.insert(tk.END, f"Challenge: {current_grid.game}\n")
        challenges_text.insert(tk.END, f"Drawn Card: {drawn_card}\n")
        challenges_text.insert(tk.END, f"Success!\nPower up: +XXX" if challenge_success else f"Failed!\nPanelty: -XXX")
        
        # Handling the other game challenges.
        """ if game2:
            if game3:
            # Update the move list text with the challenge and outcome.
            challenges_text.insert(tk.END, f"challenge: {current_grid.game}\n")
            challenges_text.insert(tk.END, f"Success!\nPower up: XXX" if challenge_success else f"Failed!\nPanelty: XXX")"""
# 5554570 - end.

# liu
def update_grid():
    """Updates the disdlplay of the game."""
    game_area.delete("all")
    for grid in grids:
        x = grid.number % 7 * 100 + 10    # Remainder = Column number.
        y = grid.number // 7 * 100 +10    # Quotient = Row number.
        create_grid(game_area, grid, x, y, player.player_position)
    game_area.pack()
# liu

# 5554570 - beggining.
player = Player(30, 30) # Player starts with 30 health points and 30 combat effectiveness.

# Main application window.
root = tk.Tk()
root.title("King Of The Hill")

# Create the game area. 
game_area = tk.Canvas(root, width=710, height=710, bg="white") 
game_area.pack(side=tk.LEFT, fill=tk.BOTH, expand=True) 

update_grid() # Draw the initial grid.

# Create a side panel for additional information. 
side_panel = tk.Frame(root, bg="light blue", width=300) 
side_panel.pack_propagate(False) # Prevents resizing
side_panel.pack(side=tk.RIGHT, fill=tk.Y, padx=10, pady=10) 

# Create a player information frame. 
player_info_frame = tk.Frame(side_panel, bg="light blue")
player_info_frame.pack(fill=tk.X)
player_label = tk.Label(player_info_frame, text="Player Info", bg="light blue")
player_label.pack()
hp_label = tk.Label(player_info_frame, text="HP: 100", bg="light blue")
hp_label.pack()
ce_label = tk.Label(player_info_frame, text="CE: 50", bg="light blue")
ce_label.pack()

# Create a frame to display the challenges and outcomes.
challenges_frame = tk.Frame(side_panel, bg="light blue")
challenges_frame.pack(fill=tk.X)
challenges_label = tk.Label(challenges_frame, text="Challenge:", bg="light blue")
challenges_label.pack()
challenges_text = tk.Text(challenges_frame, height=10, width=30)
challenges_text.pack()

# Creates a game control frame.
controls_frame = tk.Frame(side_panel, bg="light blue")
controls_frame.pack(fill=tk.X)
roll_button = tk.Button(controls_frame, text="Roll Dice", command=roll_and_move)
roll_button.pack()
dice_result = tk.Label(controls_frame, text="Roll: ", bg="light blue")
dice_result.pack()
quit_button = tk.Button(controls_frame, text="Quit", command=root.destroy)
quit_button.pack()
# 5554570 - end.

# Runs the game.
root.mainloop()